In [28]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/damoncrockett/ivpytest/src')
from ivpy import *
from ivpy.extract import extract,norm
from ivpy.utils import resize
import glob,os
import warnings
warnings.filterwarnings('ignore')

In [29]:
f = '/Users/damoncrockett/Desktop/pasfa.csv'
xf = pd.read_csv(f)

### Matching sort order of SP model

In [30]:
f = '/Users/damoncrockett/Desktop/textureparams2.csv'
mf = pd.read_csv(f,header=None)

In [31]:
xf['fullname'] = [os.path.basename(item)[:-4] for item in xf.localpath]

In [32]:
xf = xf.set_index('fullname')

In [33]:
xf = xf.loc[mf[0]]

In [34]:
xf = xf.reset_index()

In [35]:
DIR = '/Users/damoncrockett/simspace/src/assets/json/img/'

In [36]:
# won't be used in the web app, but useful for futzing with the images prior to

xf['localpath'] = [DIR + item + '.jpeg' for item in xf['fullname']]

### Creating localhost paths

In [37]:
xf['localhostpath'] = ['http://localhost:8888/img/' + os.path.basename(item) for item in xf.localpath]

### Metadata

In [38]:
xf['edition'] = [item.split('_')[0] for item in xf['fullname']]
xf['leaf'] = ['_'.join(item.split('_')[:-1]) for item in xf['fullname']]

### Creation of simspaces

The target problem is visualizing the similarity spaces of different texture models. In order to visualize these spaces, we need dimension reduction (to 2 dimensions). For now, we'll just use t-SNE and UMAP, since they are different simspaces and can return 2-dimensional display coordinates.

In [39]:
from ivpy.reduce import pca,tsne,umap

In [40]:
X = xf.iloc[:,1:9]

In [41]:
xf[['xp','yp']] = norm(pca(X,n_components=2))
xf[['xt','yt']] = norm(tsne(X))
xf[['xu','yu']] = norm(umap(X))

### JSON creation

In [42]:
import json

In [43]:
def row2jso(xf,xcol,ycol,i):
    row = xf.loc[i]
    jso = {}
    jso['imgpath'] = row.localhostpath
    jso['x'] = row[xcol]
    jso['y'] = row[ycol]
    jso['edition'] = row.edition
    jso['leaf'] = row.leaf
    jso['fullname'] = row.fullname

    return jso

In [44]:
pcaArray = [row2jso(xf,'xp','yp',i) for i in xf.index]
tsneArray = [row2jso(xf,'xt','yt',i) for i in xf.index]
umapArray = [row2jso(xf,'xu','yu',i) for i in xf.index]

In [45]:
JSONDIR = '/Users/damoncrockett/simspace/src/assets/json/'

In [46]:
with open(JSONDIR+"pasfa_pca.json",'w') as outfile:
    json.dump(pcaArray,outfile)

with open(JSONDIR+"pasfa_tsne.json",'w') as outfile:
    json.dump(tsneArray,outfile)

with open(JSONDIR+"pasfa_umap.json",'w') as outfile:
    json.dump(umapArray,outfile)